In [1]:
import numpy as np
import pandas as pd
import torch
import re
import nltk
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.preprocessing import FunctionTransformer
from sklearn.model_selection import train_test_split
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from nltk.corpus import wordnet as wn
import tensorflow
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# load the new content features data from csv.
df = pd.read_csv('/content/drive/MyDrive/School/Term 8/AI Project/Final Weights/WELFake_Dataset.csv')
df = df.dropna()
print(df.columns)
print(df.shape)
print(type(df.loc[7,"text"]))

df['merged'] = df['title'] + df['text']
merged_iterator = df['merged']

Index(['Unnamed: 0', 'title', 'text', 'label'], dtype='object')
(71537, 4)
<class 'str'>


In [4]:
import tensorflow as tf
y = df["label"].values

vocab_size = 2000

# create vectoriser
vectorizer = tf.keras.layers.TextVectorization(max_tokens=vocab_size,output_mode='tf_idf')
vectorizer.adapt(merged_iterator.values)

# create tf-idf
x = vectorizer(merged_iterator.values).numpy()
print(x.shape)

(71537, 2000)


In [5]:
# split the data
from sklearn.model_selection import train_test_split
X_train, X_temp, y_train, y_temp = train_test_split(x, y, test_size=0.2, random_state=42)
# 40 train1, 40 train2, 10 val, 10 test
X_train1, X_train2, y_train1, y_train2 = train_test_split(X_train, y_train, test_size=0.5, random_state=42)
# Further split the remaining data (20%) into validation and testing sets (50% each)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

print(X_train1.shape)
print((X_train2.shape))
print((X_val.shape))
print((X_test.shape))


(28614, 2000)
(28615, 2000)
(7154, 2000)
(7154, 2000)


In [6]:
input=X_train1.shape[1]
print(input)

2000


This is the best config

In [10]:
from keras.optimizers import Adamax
from keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

ffnn_neurons = 1024
dropout = 0.5

# Define FFNN model without embedding layer
ffnn_model = Sequential()
ffnn_model.add(Dense(ffnn_neurons, activation='relu', input_shape=(input,)))
ffnn_model.add(Dropout(dropout))
ffnn_model.add(Dense(1, activation='sigmoid'))


checkpoint_callback = ModelCheckpoint(
    filepath="/content/drive/MyDrive/School/Term 8/AI Project/bow_ffnn_model_best_new.h5",
    monitor='val_accuracy', # "val_loss" or 'val_accuracy' depending on your task
    verbose=1,
    save_best_only=True,
    mode='max' # Use 'min' for loss, 'max' for accuracy
)

# Compile the model
ffnn_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Assuming X_train and y_train1 are defined elsewhere
# Train the model
history = ffnn_model.fit(X_train1, y_train1, epochs=20, batch_size=32, validation_data=(X_val, y_val),callbacks=checkpoint_callback)

Epoch 1/20
895/895 [==============================] - ETA: 0s - loss: 0.6799 - accuracy: 0.9043
Epoch 1: val_accuracy improved from -inf to 0.95387, saving model to /content/drive/MyDrive/School/Term 8/AI Project/bow_ffnn_model_best_new.h5
895/895 [==============================] - 7s 6ms/step - loss: 0.6799 - accuracy: 0.9043 - val_loss: 0.1679 - val_accuracy: 0.9539
Epoch 2/20
890/895 [============================>.] - ETA: 0s - loss: 0.1691 - accuracy: 0.9538
Epoch 2: val_accuracy improved from 0.95387 to 0.95807, saving model to /content/drive/MyDrive/School/Term 8/AI Project/bow_ffnn_model_best_new.h5
895/895 [==============================] - 4s 4ms/step - loss: 0.1690 - accuracy: 0.9538 - val_loss: 0.1286 - val_accuracy: 0.9581
Epoch 3/20
887/895 [============================>.] - ETA: 0s - loss: 0.1070 - accuracy: 0.9683
Epoch 3: val_accuracy improved from 0.95807 to 0.96296, saving model to /content/drive/MyDrive/School/Term 8/AI Project/bow_ffnn_model_best_new.h5
895/895 [===

In [13]:
# Load the model
import tensorflow as tf
from tensorflow.keras.models import load_model

model = load_model("/content/drive/MyDrive/School/Term 8/AI Project/bow_ffnn_model_best_new.h5")
loss, acc = model.evaluate(X_test, y_test, verbose=2)
print("Model accuracy: {:5.5f}%".format(100 * acc))


224/224 - 1s - loss: 0.3457 - accuracy: 0.9639 - 526ms/epoch - 2ms/step
Model accuracy: 96.39363%


below are the testing from model tuning

In [10]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

ffnn_neurons = 128
dropout = 0.5

# Define FFNN model without embedding layer
ffnn_model = Sequential()
ffnn_model.add(Dense(ffnn_neurons, activation='sigmoid', input_shape=(input,)))
ffnn_model.add(Dropout(dropout))
ffnn_model.add(Dense(ffnn_neurons, activation='sigmoid'))
ffnn_model.add(Dropout(dropout))
ffnn_model.add(Dense(ffnn_neurons, activation='sigmoid'))
ffnn_model.add(Dropout(dropout))
ffnn_model.add(Dense(1, activation='sigmoid'))

# Compile the model
ffnn_model.compile(optimizer='Adam', loss='binary_crossentropy', metrics=['accuracy'])

# Assuming X_train and y_train1 are defined elsewhere
# Train the model
history = ffnn_model.fit(X_train1, y_train1, epochs=10, batch_size=32, validation_data=(X_val, y_val))

Epoch 1/10
895/895 [==============================] - 11s 10ms/step - loss: 0.2883 - accuracy: 0.8642 - val_loss: 0.1336 - val_accuracy: 0.9495
Epoch 2/10
652/895 [====================>.........] - ETA: 0s - loss: 0.1523 - accuracy: 0.9439

KeyboardInterrupt: 

In [8]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

ffnn_neurons = 128
dropout = 0.5

# Define FFNN model without embedding layer
ffnn_model = Sequential()
ffnn_model.add(Dense(ffnn_neurons, activation='relu', input_shape=(input,)))
ffnn_model.add(Dropout(dropout))
ffnn_model.add(Dense(ffnn_neurons, activation='relu'))
ffnn_model.add(Dropout(dropout))
ffnn_model.add(Dense(ffnn_neurons, activation='relu'))
ffnn_model.add(Dropout(dropout))
ffnn_model.add(Dense(1, activation='sigmoid'))

# Compile the model
ffnn_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Assuming X_train and y_train1 are defined elsewhere
# Train the model
history = ffnn_model.fit(X_train1, y_train1, epochs=10, batch_size=32, validation_data=(X_val, y_val))

Epoch 1/10
895/895 [==============================] - 6s 5ms/step - loss: 0.7460 - accuracy: 0.7757 - val_loss: 0.1797 - val_accuracy: 0.9356
Epoch 2/10
895/895 [==============================] - 5s 6ms/step - loss: 0.2139 - accuracy: 0.9244 - val_loss: 0.1383 - val_accuracy: 0.9469
Epoch 3/10
895/895 [==============================] - 4s 4ms/step - loss: 0.1785 - accuracy: 0.9375 - val_loss: 0.1256 - val_accuracy: 0.9535
Epoch 4/10
895/895 [==============================] - 4s 5ms/step - loss: 0.1490 - accuracy: 0.9464 - val_loss: 0.1213 - val_accuracy: 0.9558
Epoch 5/10
895/895 [==============================] - 5s 6ms/step - loss: 0.1383 - accuracy: 0.9514 - val_loss: 0.1180 - val_accuracy: 0.9588
Epoch 6/10
895/895 [==============================] - 4s 4ms/step - loss: 0.1289 - accuracy: 0.9545 - val_loss: 0.1211 - val_accuracy: 0.9526
Epoch 7/10
895/895 [==============================] - 4s 4ms/step - loss: 0.1267 - accuracy: 0.9581 - val_loss: 0.1151 - val_accuracy: 0.9572
Epoch 

In [9]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

ffnn_neurons = 128
dropout = 0.5

# Define FFNN model without embedding layer
ffnn_model = Sequential()
ffnn_model.add(Dense(ffnn_neurons, activation='Softmax', input_shape=(input,)))
ffnn_model.add(Dropout(dropout))
ffnn_model.add(Dense(ffnn_neurons, activation='Softmax'))
ffnn_model.add(Dropout(dropout))
ffnn_model.add(Dense(ffnn_neurons, activation='Softmax'))
ffnn_model.add(Dropout(dropout))
ffnn_model.add(Dense(1, activation='sigmoid'))

# Compile the model
ffnn_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Assuming X_train and y_train1 are defined elsewhere
# Train the model
history = ffnn_model.fit(X_train1, y_train1, epochs=10, batch_size=32, validation_data=(X_val, y_val))


Epoch 1/10
895/895 [==============================] - 6s 5ms/step - loss: 0.6816 - accuracy: 0.6113 - val_loss: 0.6107 - val_accuracy: 0.9421
Epoch 2/10
895/895 [==============================] - 4s 4ms/step - loss: 0.5850 - accuracy: 0.7159 - val_loss: 0.3905 - val_accuracy: 0.9451
Epoch 3/10
895/895 [==============================] - 5s 6ms/step - loss: 0.5178 - accuracy: 0.7233 - val_loss: 0.3038 - val_accuracy: 0.9400
Epoch 4/10
895/895 [==============================] - 4s 5ms/step - loss: 0.4996 - accuracy: 0.7219 - val_loss: 0.2676 - val_accuracy: 0.9407
Epoch 5/10
895/895 [==============================] - 4s 5ms/step - loss: 0.4702 - accuracy: 0.7431 - val_loss: 0.2360 - val_accuracy: 0.9448
Epoch 6/10
895/895 [==============================] - 5s 6ms/step - loss: 0.4626 - accuracy: 0.7458 - val_loss: 0.2346 - val_accuracy: 0.9428
Epoch 7/10
895/895 [==============================] - 4s 5ms/step - loss: 0.4388 - accuracy: 0.7689 - val_loss: 0.2506 - val_accuracy: 0.9304
Epoch 

In [11]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

ffnn_neurons = 128
dropout = 0.5

# Define FFNN model without embedding layer
ffnn_model = Sequential()
ffnn_model.add(Dense(ffnn_neurons, activation='relu', input_shape=(input,)))
ffnn_model.add(Dropout(dropout))
ffnn_model.add(Dense(ffnn_neurons, activation='relu'))
ffnn_model.add(Dropout(dropout))
ffnn_model.add(Dense(ffnn_neurons, activation='relu'))
ffnn_model.add(Dropout(dropout))
ffnn_model.add(Dense(1, activation='sigmoid'))

# Compile the model
ffnn_model.compile(optimizer='adagrad', loss='binary_crossentropy', metrics=['accuracy'])

# Assuming X_train and y_train1 are defined elsewhere
# Train the model
history = ffnn_model.fit(X_train1, y_train1, epochs=10, batch_size=32, validation_data=(X_val, y_val))


Epoch 1/10
895/895 [==============================] - 6s 6ms/step - loss: 3.7159 - accuracy: 0.5092 - val_loss: 0.6590 - val_accuracy: 0.6314
Epoch 2/10
895/895 [==============================] - 4s 4ms/step - loss: 2.0039 - accuracy: 0.5172 - val_loss: 0.6635 - val_accuracy: 0.6356
Epoch 3/10
895/895 [==============================] - 4s 4ms/step - loss: 1.5557 - accuracy: 0.5199 - val_loss: 0.6590 - val_accuracy: 0.6533
Epoch 4/10
895/895 [==============================] - 5s 6ms/step - loss: 1.3178 - accuracy: 0.5322 - val_loss: 0.6588 - val_accuracy: 0.6696
Epoch 5/10
895/895 [==============================] - 4s 4ms/step - loss: 1.1406 - accuracy: 0.5338 - val_loss: 0.6609 - val_accuracy: 0.6670
Epoch 6/10
895/895 [==============================] - 4s 4ms/step - loss: 1.0305 - accuracy: 0.5447 - val_loss: 0.6613 - val_accuracy: 0.6743
Epoch 7/10
895/895 [==============================] - 5s 5ms/step - loss: 0.9643 - accuracy: 0.5552 - val_loss: 0.6565 - val_accuracy: 0.6827
Epoch 

In [12]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

ffnn_neurons = 128
dropout = 0.5

# Define FFNN model without embedding layer
ffnn_model = Sequential()
ffnn_model.add(Dense(ffnn_neurons, activation='relu', input_shape=(input,)))
ffnn_model.add(Dropout(dropout))
ffnn_model.add(Dense(ffnn_neurons, activation='relu'))
ffnn_model.add(Dropout(dropout))
ffnn_model.add(Dense(ffnn_neurons, activation='relu'))
ffnn_model.add(Dropout(dropout))
ffnn_model.add(Dense(1, activation='sigmoid'))

# Compile the model
ffnn_model.compile(optimizer='adamax', loss='binary_crossentropy', metrics=['accuracy'])

# Assuming X_train and y_train1 are defined elsewhere
# Train the model
history = ffnn_model.fit(X_train1, y_train1, epochs=10, batch_size=32, validation_data=(X_val, y_val))


Epoch 1/10
895/895 [==============================] - 6s 5ms/step - loss: 2.4034 - accuracy: 0.5359 - val_loss: 0.6528 - val_accuracy: 0.7006
Epoch 2/10
895/895 [==============================] - 5s 5ms/step - loss: 0.7030 - accuracy: 0.6898 - val_loss: 0.3930 - val_accuracy: 0.9125
Epoch 3/10
895/895 [==============================] - 4s 5ms/step - loss: 0.3992 - accuracy: 0.8530 - val_loss: 0.2308 - val_accuracy: 0.9272
Epoch 4/10
895/895 [==============================] - 4s 4ms/step - loss: 0.2843 - accuracy: 0.9018 - val_loss: 0.1736 - val_accuracy: 0.9386
Epoch 5/10
895/895 [==============================] - 5s 5ms/step - loss: 0.2340 - accuracy: 0.9171 - val_loss: 0.1563 - val_accuracy: 0.9431
Epoch 6/10
895/895 [==============================] - 4s 5ms/step - loss: 0.2060 - accuracy: 0.9290 - val_loss: 0.1469 - val_accuracy: 0.9427
Epoch 7/10
895/895 [==============================] - 4s 4ms/step - loss: 0.1788 - accuracy: 0.9380 - val_loss: 0.1362 - val_accuracy: 0.9484
Epoch 

In [13]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

ffnn_neurons = 256
dropout = 0.5

# Define FFNN model without embedding layer
ffnn_model = Sequential()
ffnn_model.add(Dense(ffnn_neurons, activation='relu', input_shape=(input,)))
ffnn_model.add(Dropout(dropout))
ffnn_model.add(Dense(ffnn_neurons, activation='relu'))
ffnn_model.add(Dropout(dropout))
ffnn_model.add(Dense(ffnn_neurons, activation='relu'))
ffnn_model.add(Dropout(dropout))
ffnn_model.add(Dense(1, activation='sigmoid'))

# Compile the model
ffnn_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Assuming X_train and y_train1 are defined elsewhere
# Train the model
history = ffnn_model.fit(X_train1, y_train1, epochs=10, batch_size=32, validation_data=(X_val, y_val))


Epoch 1/10
895/895 [==============================] - 5s 4ms/step - loss: 0.8720 - accuracy: 0.8038 - val_loss: 0.1772 - val_accuracy: 0.9361
Epoch 2/10
895/895 [==============================] - 4s 5ms/step - loss: 0.2054 - accuracy: 0.9303 - val_loss: 0.1407 - val_accuracy: 0.9522
Epoch 3/10
895/895 [==============================] - 5s 6ms/step - loss: 0.1572 - accuracy: 0.9447 - val_loss: 0.1297 - val_accuracy: 0.9514
Epoch 4/10
895/895 [==============================] - 4s 5ms/step - loss: 0.1385 - accuracy: 0.9517 - val_loss: 0.1168 - val_accuracy: 0.9579
Epoch 5/10
895/895 [==============================] - 4s 5ms/step - loss: 0.1181 - accuracy: 0.9588 - val_loss: 0.1170 - val_accuracy: 0.9554
Epoch 6/10
895/895 [==============================] - 5s 6ms/step - loss: 0.1186 - accuracy: 0.9582 - val_loss: 0.1148 - val_accuracy: 0.9597
Epoch 7/10
895/895 [==============================] - 4s 4ms/step - loss: 0.1042 - accuracy: 0.9630 - val_loss: 0.1063 - val_accuracy: 0.9625
Epoch 

In [14]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

ffnn_neurons = 256
dropout = 0.1

# Define FFNN model without embedding layer
ffnn_model = Sequential()
ffnn_model.add(Dense(ffnn_neurons, activation='relu', input_shape=(input,)))
ffnn_model.add(Dropout(dropout))
ffnn_model.add(Dense(ffnn_neurons, activation='relu'))
ffnn_model.add(Dropout(dropout))
ffnn_model.add(Dense(ffnn_neurons, activation='relu'))
ffnn_model.add(Dropout(dropout))
ffnn_model.add(Dense(1, activation='sigmoid'))

# Compile the model
ffnn_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Assuming X_train and y_train1 are defined elsewhere
# Train the model
history = ffnn_model.fit(X_train1, y_train1, epochs=10, batch_size=32, validation_data=(X_val, y_val))


Epoch 1/10
895/895 [==============================] - 7s 6ms/step - loss: 0.3552 - accuracy: 0.8931 - val_loss: 0.1330 - val_accuracy: 0.9530
Epoch 2/10
895/895 [==============================] - 4s 5ms/step - loss: 0.1326 - accuracy: 0.9546 - val_loss: 0.1230 - val_accuracy: 0.9586
Epoch 3/10
895/895 [==============================] - 4s 5ms/step - loss: 0.1038 - accuracy: 0.9619 - val_loss: 0.1341 - val_accuracy: 0.9611
Epoch 4/10
895/895 [==============================] - 5s 6ms/step - loss: 0.0861 - accuracy: 0.9691 - val_loss: 0.1275 - val_accuracy: 0.9602
Epoch 5/10
895/895 [==============================] - 4s 5ms/step - loss: 0.0694 - accuracy: 0.9754 - val_loss: 0.1297 - val_accuracy: 0.9606
Epoch 6/10
895/895 [==============================] - 4s 4ms/step - loss: 0.0554 - accuracy: 0.9800 - val_loss: 0.1235 - val_accuracy: 0.9635
Epoch 7/10
895/895 [==============================] - 5s 6ms/step - loss: 0.0505 - accuracy: 0.9828 - val_loss: 0.1496 - val_accuracy: 0.9637
Epoch 

In [15]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

ffnn_neurons = 256
dropout = 0.05

# Define FFNN model without embedding layer
ffnn_model = Sequential()
ffnn_model.add(Dense(ffnn_neurons, activation='relu', input_shape=(input,)))
ffnn_model.add(Dropout(dropout))
ffnn_model.add(Dense(ffnn_neurons, activation='relu'))
ffnn_model.add(Dropout(dropout))
ffnn_model.add(Dense(ffnn_neurons, activation='relu'))
ffnn_model.add(Dropout(dropout))
ffnn_model.add(Dense(1, activation='sigmoid'))

# Compile the model
ffnn_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Assuming X_train and y_train1 are defined elsewhere
# Train the model
history = ffnn_model.fit(X_train1, y_train1, epochs=10, batch_size=32, validation_data=(X_val, y_val))


Epoch 1/10
895/895 [==============================] - 5s 5ms/step - loss: 0.3276 - accuracy: 0.9039 - val_loss: 0.1509 - val_accuracy: 0.9507
Epoch 2/10
895/895 [==============================] - 4s 5ms/step - loss: 0.1330 - accuracy: 0.9532 - val_loss: 0.1222 - val_accuracy: 0.9567
Epoch 3/10
895/895 [==============================] - 5s 6ms/step - loss: 0.0954 - accuracy: 0.9667 - val_loss: 0.1390 - val_accuracy: 0.9571
Epoch 4/10
895/895 [==============================] - 4s 5ms/step - loss: 0.0809 - accuracy: 0.9712 - val_loss: 0.1282 - val_accuracy: 0.9625
Epoch 5/10
895/895 [==============================] - 4s 4ms/step - loss: 0.0636 - accuracy: 0.9770 - val_loss: 0.1378 - val_accuracy: 0.9628
Epoch 6/10
895/895 [==============================] - 5s 6ms/step - loss: 0.0496 - accuracy: 0.9820 - val_loss: 0.1655 - val_accuracy: 0.9656
Epoch 7/10
895/895 [==============================] - 4s 4ms/step - loss: 0.0505 - accuracy: 0.9829 - val_loss: 0.1607 - val_accuracy: 0.9623
Epoch 

In [16]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

ffnn_neurons = 256
dropout = 0.5

# Define FFNN model without embedding layer
ffnn_model = Sequential()
ffnn_model.add(Dense(ffnn_neurons, activation='relu', input_shape=(input,)))
ffnn_model.add(Dropout(dropout))
ffnn_model.add(Dense(ffnn_neurons, activation='relu'))
ffnn_model.add(Dropout(dropout))
ffnn_model.add(Dense(ffnn_neurons, activation='relu'))
ffnn_model.add(Dropout(dropout))
ffnn_model.add(Dense(ffnn_neurons, activation='relu'))
ffnn_model.add(Dropout(dropout))
ffnn_model.add(Dense(ffnn_neurons, activation='relu'))
ffnn_model.add(Dropout(dropout))
ffnn_model.add(Dense(1, activation='sigmoid'))

# Compile the model
ffnn_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Assuming X_train and y_train1 are defined elsewhere
# Train the model
history = ffnn_model.fit(X_train1, y_train1, epochs=10, batch_size=32, validation_data=(X_val, y_val))


Epoch 1/10
895/895 [==============================] - 7s 5ms/step - loss: 1.0709 - accuracy: 0.5253 - val_loss: 0.6673 - val_accuracy: 0.6621
Epoch 2/10
895/895 [==============================] - 5s 6ms/step - loss: 0.3660 - accuracy: 0.8587 - val_loss: 0.1452 - val_accuracy: 0.9466
Epoch 3/10
895/895 [==============================] - 5s 5ms/step - loss: 0.1937 - accuracy: 0.9364 - val_loss: 0.1302 - val_accuracy: 0.9511
Epoch 4/10
895/895 [==============================] - 5s 6ms/step - loss: 0.1532 - accuracy: 0.9482 - val_loss: 0.1299 - val_accuracy: 0.9529
Epoch 5/10
895/895 [==============================] - 5s 6ms/step - loss: 0.1325 - accuracy: 0.9558 - val_loss: 0.1358 - val_accuracy: 0.9484
Epoch 6/10
895/895 [==============================] - 5s 5ms/step - loss: 0.1252 - accuracy: 0.9574 - val_loss: 0.1090 - val_accuracy: 0.9614
Epoch 7/10
895/895 [==============================] - 5s 6ms/step - loss: 0.1163 - accuracy: 0.9627 - val_loss: 0.1151 - val_accuracy: 0.9607
Epoch 

In [17]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

ffnn_neurons = 256
dropout = 0.5

# Define FFNN model without embedding layer
ffnn_model = Sequential()
ffnn_model.add(Dense(ffnn_neurons, activation='relu', input_shape=(input,)))
ffnn_model.add(Dropout(dropout))
ffnn_model.add(Dense(1, activation='sigmoid'))

# Compile the model
ffnn_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Assuming X_train and y_train1 are defined elsewhere
# Train the model
history = ffnn_model.fit(X_train1, y_train1, epochs=10, batch_size=32, validation_data=(X_val, y_val))


Epoch 1/10
895/895 [==============================] - 6s 5ms/step - loss: 0.7651 - accuracy: 0.8843 - val_loss: 0.1685 - val_accuracy: 0.9420
Epoch 2/10
895/895 [==============================] - 4s 4ms/step - loss: 0.1632 - accuracy: 0.9449 - val_loss: 0.1265 - val_accuracy: 0.9529
Epoch 3/10
895/895 [==============================] - 3s 4ms/step - loss: 0.1354 - accuracy: 0.9546 - val_loss: 0.1231 - val_accuracy: 0.9562
Epoch 4/10
895/895 [==============================] - 3s 4ms/step - loss: 0.1066 - accuracy: 0.9635 - val_loss: 0.1511 - val_accuracy: 0.9483
Epoch 5/10
895/895 [==============================] - 4s 5ms/step - loss: 0.1151 - accuracy: 0.9629 - val_loss: 0.1814 - val_accuracy: 0.9358
Epoch 6/10
895/895 [==============================] - 3s 4ms/step - loss: 0.1018 - accuracy: 0.9659 - val_loss: 0.1285 - val_accuracy: 0.9576
Epoch 7/10
895/895 [==============================] - 3s 4ms/step - loss: 0.0848 - accuracy: 0.9699 - val_loss: 0.1243 - val_accuracy: 0.9603
Epoch 

In [18]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

ffnn_neurons = 512
dropout = 0.5

# Define FFNN model without embedding layer
ffnn_model = Sequential()
ffnn_model.add(Dense(ffnn_neurons, activation='relu', input_shape=(input,)))
ffnn_model.add(Dropout(dropout))
ffnn_model.add(Dense(1, activation='sigmoid'))

# Compile the model
ffnn_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Assuming X_train and y_train1 are defined elsewhere
# Train the model
history = ffnn_model.fit(X_train1, y_train1, epochs=10, batch_size=32, validation_data=(X_val, y_val))


Epoch 1/10
895/895 [==============================] - 6s 4ms/step - loss: 0.6981 - accuracy: 0.8931 - val_loss: 0.1572 - val_accuracy: 0.9505
Epoch 2/10
895/895 [==============================] - 3s 4ms/step - loss: 0.1675 - accuracy: 0.9483 - val_loss: 0.1398 - val_accuracy: 0.9526
Epoch 3/10
895/895 [==============================] - 3s 4ms/step - loss: 0.1403 - accuracy: 0.9543 - val_loss: 0.1167 - val_accuracy: 0.9593
Epoch 4/10
895/895 [==============================] - 4s 5ms/step - loss: 0.1174 - accuracy: 0.9607 - val_loss: 0.1105 - val_accuracy: 0.9634
Epoch 5/10
895/895 [==============================] - 3s 4ms/step - loss: 0.1099 - accuracy: 0.9649 - val_loss: 0.1132 - val_accuracy: 0.9599
Epoch 6/10
895/895 [==============================] - 3s 4ms/step - loss: 0.0911 - accuracy: 0.9702 - val_loss: 0.1032 - val_accuracy: 0.9637
Epoch 7/10
895/895 [==============================] - 4s 4ms/step - loss: 0.1028 - accuracy: 0.9673 - val_loss: 0.1050 - val_accuracy: 0.9676
Epoch 

In [19]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

ffnn_neurons = 512
dropout = 0.5

# Define FFNN model without embedding layer
ffnn_model = Sequential()
ffnn_model.add(Dense(ffnn_neurons, activation='relu', input_shape=(input,)))
ffnn_model.add(Dropout(dropout))
ffnn_model.add(Dense(ffnn_neurons, activation='relu'))
ffnn_model.add(Dropout(dropout))
ffnn_model.add(Dense(ffnn_neurons, activation='relu'))
ffnn_model.add(Dropout(dropout))
ffnn_model.add(Dense(ffnn_neurons, activation='relu'))
ffnn_model.add(Dropout(dropout))
ffnn_model.add(Dense(ffnn_neurons, activation='relu'))
ffnn_model.add(Dropout(dropout))
ffnn_model.add(Dense(1, activation='sigmoid'))

# Compile the model
ffnn_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Assuming X_train and y_train1 are defined elsewhere
# Train the model
history = ffnn_model.fit(X_train1, y_train1, epochs=10, batch_size=32, validation_data=(X_val, y_val))


Epoch 1/10
895/895 [==============================] - 7s 5ms/step - loss: 1.1333 - accuracy: 0.6501 - val_loss: 0.2274 - val_accuracy: 0.9172
Epoch 2/10
895/895 [==============================] - 5s 6ms/step - loss: 0.2345 - accuracy: 0.9220 - val_loss: 0.1599 - val_accuracy: 0.9488
Epoch 3/10
895/895 [==============================] - 5s 5ms/step - loss: 0.1664 - accuracy: 0.9418 - val_loss: 0.1203 - val_accuracy: 0.9553
Epoch 4/10
895/895 [==============================] - 5s 5ms/step - loss: 0.1460 - accuracy: 0.9500 - val_loss: 0.1351 - val_accuracy: 0.9558
Epoch 5/10
895/895 [==============================] - 6s 6ms/step - loss: 0.1352 - accuracy: 0.9540 - val_loss: 0.1350 - val_accuracy: 0.9572
Epoch 6/10
895/895 [==============================] - 5s 5ms/step - loss: 0.1270 - accuracy: 0.9569 - val_loss: 0.1646 - val_accuracy: 0.9555
Epoch 7/10
895/895 [==============================] - 5s 6ms/step - loss: 0.1264 - accuracy: 0.9594 - val_loss: 0.1248 - val_accuracy: 0.9575
Epoch 

In [20]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

ffnn_neurons = 512
dropout = 0.5

# Define FFNN model without embedding layer
ffnn_model = Sequential()
ffnn_model.add(Dense(ffnn_neurons, activation='relu', input_shape=(input,)))
ffnn_model.add(Dropout(dropout))
ffnn_model.add(Dense(ffnn_neurons, activation='relu'))
ffnn_model.add(Dropout(dropout))
ffnn_model.add(Dense(ffnn_neurons, activation='relu'))
ffnn_model.add(Dropout(dropout))
ffnn_model.add(Dense(1, activation='sigmoid'))

# Compile the model
ffnn_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Assuming X_train and y_train1 are defined elsewhere
# Train the model
history = ffnn_model.fit(X_train1, y_train1, epochs=10, batch_size=32, validation_data=(X_val, y_val))


Epoch 1/10
895/895 [==============================] - 6s 5ms/step - loss: 0.9118 - accuracy: 0.8226 - val_loss: 0.1861 - val_accuracy: 0.9441
Epoch 2/10
895/895 [==============================] - 5s 6ms/step - loss: 0.2041 - accuracy: 0.9337 - val_loss: 0.1379 - val_accuracy: 0.9529
Epoch 3/10
895/895 [==============================] - 4s 5ms/step - loss: 0.1522 - accuracy: 0.9483 - val_loss: 0.1471 - val_accuracy: 0.9467
Epoch 4/10
895/895 [==============================] - 4s 5ms/step - loss: 0.1348 - accuracy: 0.9544 - val_loss: 0.1247 - val_accuracy: 0.9535
Epoch 5/10
895/895 [==============================] - 5s 6ms/step - loss: 0.1283 - accuracy: 0.9558 - val_loss: 0.1203 - val_accuracy: 0.9547
Epoch 6/10
895/895 [==============================] - 4s 5ms/step - loss: 0.1062 - accuracy: 0.9627 - val_loss: 0.1240 - val_accuracy: 0.9600
Epoch 7/10
895/895 [==============================] - 4s 5ms/step - loss: 0.1177 - accuracy: 0.9608 - val_loss: 0.1299 - val_accuracy: 0.9498
Epoch 

In [21]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

ffnn_neurons = 1024
dropout = 0.5

# Define FFNN model without embedding layer
ffnn_model = Sequential()
ffnn_model.add(Dense(ffnn_neurons, activation='relu', input_shape=(input,)))
ffnn_model.add(Dropout(dropout))
ffnn_model.add(Dense(ffnn_neurons, activation='relu'))
ffnn_model.add(Dropout(dropout))
ffnn_model.add(Dense(ffnn_neurons, activation='relu'))
ffnn_model.add(Dropout(dropout))
ffnn_model.add(Dense(1, activation='sigmoid'))

# Compile the model
ffnn_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Assuming X_train and y_train1 are defined elsewhere
# Train the model
history = ffnn_model.fit(X_train1, y_train1, epochs=10, batch_size=32, validation_data=(X_val, y_val))


Epoch 1/10
895/895 [==============================] - 7s 6ms/step - loss: 1.0919 - accuracy: 0.8629 - val_loss: 0.1660 - val_accuracy: 0.9425
Epoch 2/10
895/895 [==============================] - 5s 5ms/step - loss: 0.2026 - accuracy: 0.9348 - val_loss: 0.1309 - val_accuracy: 0.9508
Epoch 3/10
895/895 [==============================] - 5s 5ms/step - loss: 0.1575 - accuracy: 0.9465 - val_loss: 0.1602 - val_accuracy: 0.9511
Epoch 4/10
895/895 [==============================] - 4s 5ms/step - loss: 0.1415 - accuracy: 0.9525 - val_loss: 0.1202 - val_accuracy: 0.9575
Epoch 5/10
895/895 [==============================] - 5s 5ms/step - loss: 0.1385 - accuracy: 0.9544 - val_loss: 0.1200 - val_accuracy: 0.9583
Epoch 6/10
895/895 [==============================] - 5s 5ms/step - loss: 0.1152 - accuracy: 0.9592 - val_loss: 0.1249 - val_accuracy: 0.9597
Epoch 7/10
895/895 [==============================] - 4s 5ms/step - loss: 0.1172 - accuracy: 0.9596 - val_loss: 0.1384 - val_accuracy: 0.9599
Epoch 

In [22]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

ffnn_neurons = 1024
dropout = 0.5

# Define FFNN model without embedding layer
ffnn_model = Sequential()
ffnn_model.add(Dense(ffnn_neurons, activation='relu', input_shape=(input,)))
ffnn_model.add(Dropout(dropout))
ffnn_model.add(Dense(1, activation='sigmoid'))

# Compile the model
ffnn_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Assuming X_train and y_train1 are defined elsewhere
# Train the model
history = ffnn_model.fit(X_train1, y_train1, epochs=10, batch_size=32, validation_data=(X_val, y_val))


Epoch 1/10
895/895 [==============================] - 5s 4ms/step - loss: 0.7905 - accuracy: 0.8917 - val_loss: 0.2108 - val_accuracy: 0.9449
Epoch 2/10
895/895 [==============================] - 4s 4ms/step - loss: 0.1987 - accuracy: 0.9437 - val_loss: 0.1374 - val_accuracy: 0.9539
Epoch 3/10
895/895 [==============================] - 4s 5ms/step - loss: 0.1561 - accuracy: 0.9530 - val_loss: 0.1271 - val_accuracy: 0.9572
Epoch 4/10
895/895 [==============================] - 4s 4ms/step - loss: 0.1454 - accuracy: 0.9555 - val_loss: 0.1220 - val_accuracy: 0.9597
Epoch 5/10
895/895 [==============================] - 3s 4ms/step - loss: 0.1257 - accuracy: 0.9600 - val_loss: 0.1341 - val_accuracy: 0.9579
Epoch 6/10
895/895 [==============================] - 4s 4ms/step - loss: 0.1619 - accuracy: 0.9605 - val_loss: 0.1988 - val_accuracy: 0.9396
Epoch 7/10
895/895 [==============================] - 4s 5ms/step - loss: 0.0864 - accuracy: 0.9720 - val_loss: 0.1249 - val_accuracy: 0.9634
Epoch 